# Skenario 1 : Dataset 1, Tanpa KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path ke file di Google Drive
file_path = '/content/drive/My Drive/FP RSBP/dataset1.csv'

# Membaca file CSV
df = pd.read_csv(file_path)
print(df.head())

         V1        V3        V4        V5        V7        V8       V10  \
0 -0.260648  2.496266 -0.083724  0.129681  0.519014 -0.130006  0.637735   
1  0.985100  0.558056 -0.429654  0.277140  0.406466 -0.133118  0.529808   
2 -0.260272  1.728538 -0.457986  0.074062  0.743511 -0.095576  0.690708   
3 -0.152152  1.746840 -1.090178  0.249486  0.518269 -0.065130  0.575231   
4 -0.206820  1.527053 -0.448293  0.106125  0.658849 -0.212660  0.968046   

        V11       V12       V13       V14       V17       V18       V19  \
0 -0.987020  0.293438 -0.941386  0.549020  0.512307  0.333644  0.124270   
1  0.140107  1.564246  0.574074  0.627719  0.403810  0.201799 -0.340687   
2 -0.272985  0.659201  0.805173  0.616874  0.886526  0.239442 -2.366079   
3 -0.752581  0.737483  0.592994  0.559535  0.242629  2.178616 -1.345060   
4 -1.203171  1.029577  1.439310  0.241454  0.366466  0.291782  0.445317   

        V23       V26       V28  Class  
0 -0.134794 -0.434824 -0.151045      0  
1  0.079469  0.2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 1. Import dataset
data_train = df

# 2. Pisahkan fitur (X) dan label (y)
X = data_train.drop('Class', axis=1)
y = data_train['Class'].astype('int64')

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Definisikan generator dan diskriminator

# Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(X_scaled.shape[1], activation='tanh'))  # Output layer with same shape as input
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output is a probability (real/fake)
    return model

# 4. Buat GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training generator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# 5. Training model AnoGAN
latent_dim = 100  # Dimensi ruang laten untuk generator

# Bangun model generator dan diskriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(X_scaled.shape[1])

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bangun dan compile GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Hyperparameters
batch_size = 64
epochs = 1000
half_batch = batch_size // 2

# Fungsi untuk training AnoGAN
for epoch in range(epochs):
    # Latih discriminator
    idx = np.random.randint(0, X_scaled.shape[0], half_batch)
    real_data = X_scaled[idx]

    # Sampel noise untuk data palsu dari generator
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_data = generator.predict(noise)

    # Label untuk data asli dan palsu
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Latih diskriminator dengan data asli dan palsu
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

    # Latih generator (membekukan diskriminator untuk sementara)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # Generator mencoba menghasilkan data yang 'valid'
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Tampilkan kemajuan pelatihan setiap 1000 epoch
    if epoch % 1000 == 0:
        print(f"{epoch}/{epochs} [D loss: {d_loss_real[0]}] [G loss: {g_loss}]")

# 6. Prediksi dan deteksi anomali
# Menghitung error rekonstruksi untuk mendeteksi anomali
reconstructed_data = generator.predict(np.random.normal(0, 1, (X_scaled.shape[0], latent_dim)))
reconstruction_error = np.mean(np.abs(reconstructed_data - X_scaled), axis=1)

# Menentukan threshold untuk mendeteksi anomali (misalnya menggunakan quantile atau manual)
threshold = np.percentile(reconstruction_error, 95)  # ambil threshold 95% dari error rekonstruksi

# Deteksi anomali
y_pred = (reconstruction_error > threshold).astype(int)

# Evaluasi performa model
print("Laporan Klasifikasi:")
print(classification_report(y, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0/1000 [D loss: 0.7081487774848938] [G loss: [array(0.7030538, dtype=float32), array(0.7030538, dtype=float32), array(0.421875, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

# Skenario 2 : Dataset 1, Dengan KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path ke file di Google Drive
file_path = '/content/drive/My Drive/FP RSBP/dataset2.csv'

# Membaca file CSV
credit_card2023 = pd.read_csv(file_path)
print(credit_card2023.head())

        V14       V10        V4       V12       V17        V3       V11  \
0 -0.311169  0.090794  1.378155 -0.617801  0.207971  2.536347 -0.551600   
1 -0.143772 -0.166974  0.448154  1.065235 -0.114805  0.166480  1.612727   
2 -0.165946  0.207643  0.379780  0.066084  1.109969  1.773209  0.624501   
3 -0.287924 -0.054952 -0.863291  0.178228 -0.684093  1.792993 -0.226487   
4 -1.119670  0.753074  0.403034  0.538196 -0.237033  1.548718 -0.822843   

        V16        V2        V9  Class  
0 -0.470401 -0.072781  0.363787      0  
1  0.463917  0.266151 -0.255425      0  
2 -2.890083 -1.340163 -1.514654      0  
3 -1.059647 -0.185226 -1.387024      0  
4 -0.451449  0.877737  0.817739      0  


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

# 1. Import dataset
data_train = credit_card2023

# 2. Pisahkan fitur (X) dan label (y)
X = data_train.drop('Class', axis=1)
y = data_train['Class'].astype('int64')

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Definisikan generator dan diskriminator

# Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(X_scaled.shape[1], activation='tanh'))  # Output layer with same shape as input
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output is a probability (real/fake)
    return model

# 4. Buat GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training generator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# 5. Training model AnoGAN
latent_dim = 100  # Dimensi ruang laten untuk generator

# Bangun model generator dan diskriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(X_scaled.shape[1])

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bangun dan compile GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Hyperparameters
batch_size = 64
epochs = 1000
half_batch = batch_size // 2

# 6. KFold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fungsi untuk training AnoGAN dengan KFold
for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
    print(f"Training fold {fold + 1}")

    # Split data berdasarkan KFold
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Training loop untuk satu fold
    for epoch in range(epochs):
        # Latih discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_data = X_train[idx]

        # Sampel noise untuk data palsu dari generator
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_data = generator.predict(noise)

        # Label untuk data asli dan palsu
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Latih diskriminator dengan data asli dan palsu
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

        # Latih generator (membekukan diskriminator untuk sementara)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))  # Generator mencoba menghasilkan data yang 'valid'
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Tampilkan kemajuan pelatihan setiap 100 epoch
        if epoch % 100 == 0:
            print(f"Fold {fold + 1} - Epoch {epoch}/{epochs} [D loss: {d_loss_real[0]}] [G loss: {g_loss}]")

    # Setelah training, lakukan prediksi dan deteksi anomali
    # Menghitung error rekonstruksi untuk mendeteksi anomali
    reconstructed_data = generator.predict(np.random.normal(0, 1, (X_val.shape[0], latent_dim)))
    reconstruction_error = np.mean(np.abs(reconstructed_data - X_val), axis=1)

    # Menentukan threshold untuk mendeteksi anomali (misalnya menggunakan quantile atau manual)
    threshold = np.percentile(reconstruction_error, 95)  # ambil threshold 95% dari error rekonstruksi

    # Deteksi anomali
    y_pred = (reconstruction_error > threshold).astype(int)

    # Evaluasi performa model pada fold ini
    print(f"Evaluasi Fold {fold + 1}:")
    print(classification_report(y_val, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training fold 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Fold 1 - Epoch 0/1000 [D loss: 0.6823797225952148] [G loss: [array(0.68181276, dtype=float32), array(0.68181276, dtype=float32), array(0.78125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

# Skenario 3 : Dataset 2, Tanpa KFold

In [ ]:
import pandas as pd

# Path ke file di Google Drive
file_path = '/content/drive/My Drive/FP RSBP/dataset2.csv'

# Membaca file CSV
credit_card2023 = pd.read_csv(file_path)
print(credit_card2023.head())

        V14       V10        V4       V12       V17        V3       V11  \
0 -0.311169  0.090794  1.378155 -0.617801  0.207971  2.536347 -0.551600   
1 -0.143772 -0.166974  0.448154  1.065235 -0.114805  0.166480  1.612727   
2 -0.165946  0.207643  0.379780  0.066084  1.109969  1.773209  0.624501   
3 -0.287924 -0.054952 -0.863291  0.178228 -0.684093  1.792993 -0.226487   
4 -1.119670  0.753074  0.403034  0.538196 -0.237033  1.548718 -0.822843   

        V16        V2        V9  Class  
0 -0.470401 -0.072781  0.363787      0  
1  0.463917  0.266151 -0.255425      0  
2 -2.890083 -1.340163 -1.514654      0  
3 -1.059647 -0.185226 -1.387024      0  
4 -0.451449  0.877737  0.817739      0  


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 1. Import dataset
data_train = credit_card2023

# 2. Pisahkan fitur (X) dan label (y)
X = data_train.drop('Class', axis=1)
y = data_train['Class'].astype('int64')

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Definisikan generator dan diskriminator

# Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(X_scaled.shape[1], activation='tanh'))  # Output layer with same shape as input
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output is a probability (real/fake)
    return model

# 4. Buat GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training generator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# 5. Training model AnoGAN
latent_dim = 100  # Dimensi ruang laten untuk generator

# Bangun model generator dan diskriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(X_scaled.shape[1])

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bangun dan compile GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Hyperparameters
batch_size = 64
epochs = 1000
half_batch = batch_size // 2

# Fungsi untuk training AnoGAN
for epoch in range(epochs):
    # Latih discriminator
    idx = np.random.randint(0, X_scaled.shape[0], half_batch)
    real_data = X_scaled[idx]

    # Sampel noise untuk data palsu dari generator
    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_data = generator.predict(noise)

    # Label untuk data asli dan palsu
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Latih diskriminator dengan data asli dan palsu
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

    # Latih generator (membekukan diskriminator untuk sementara)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # Generator mencoba menghasilkan data yang 'valid'
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Tampilkan kemajuan pelatihan setiap 1000 epoch
    if epoch % 1000 == 0:
        print(f"{epoch}/{epochs} [D loss: {d_loss_real[0]}] [G loss: {g_loss}]")

# 6. Prediksi dan deteksi anomali
# Menghitung error rekonstruksi untuk mendeteksi anomali
reconstructed_data = generator.predict(np.random.normal(0, 1, (X_scaled.shape[0], latent_dim)))
reconstruction_error = np.mean(np.abs(reconstructed_data - X_scaled), axis=1)

# Menentukan threshold untuk mendeteksi anomali (misalnya menggunakan quantile atau manual)
threshold = np.percentile(reconstruction_error, 95)  # ambil threshold 95% dari error rekonstruksi

# Deteksi anomali
y_pred = (reconstruction_error > threshold).astype(int)

# Evaluasi performa model
print("Laporan Klasifikasi:")
print(classification_report(y, y_pred))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0/1000 [D loss: 0.6317973136901855] [G loss: [array(0.66834575, dtype=float32), array(0.66834575, dtype=float32), array(0.5625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

# Skenario 4 : Dataset 2, Dengan KFold

In [ ]:
import pandas as pd

# Path ke file di Google Drive
file_path = '/content/drive/My Drive/FP RSBP/dataset2.csv'

# Membaca file CSV
credit_card2023 = pd.read_csv(file_path)
print(credit_card2023.head())

        V14       V10        V4       V12       V17        V3       V11  \
0 -0.311169  0.090794  1.378155 -0.617801  0.207971  2.536347 -0.551600   
1 -0.143772 -0.166974  0.448154  1.065235 -0.114805  0.166480  1.612727   
2 -0.165946  0.207643  0.379780  0.066084  1.109969  1.773209  0.624501   
3 -0.287924 -0.054952 -0.863291  0.178228 -0.684093  1.792993 -0.226487   
4 -1.119670  0.753074  0.403034  0.538196 -0.237033  1.548718 -0.822843   

        V16        V2        V9  Class  
0 -0.470401 -0.072781  0.363787      0  
1  0.463917  0.266151 -0.255425      0  
2 -2.890083 -1.340163 -1.514654      0  
3 -1.059647 -0.185226 -1.387024      0  
4 -0.451449  0.877737  0.817739      0  


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

# 1. Import dataset
data_train = credit_card2023

# 2. Pisahkan fitur (X) dan label (y)
X = data_train.drop('Class', axis=1)
y = data_train['Class'].astype('int64')

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Definisikan generator dan diskriminator

# Generator model
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(X_scaled.shape[1], activation='tanh'))  # Output layer with same shape as input
    return model

# Discriminator model
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output is a probability (real/fake)
    return model

# 4. Buat GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training generator
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# 5. Training model AnoGAN
latent_dim = 100  # Dimensi ruang laten untuk generator

# Bangun model generator dan diskriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(X_scaled.shape[1])

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bangun dan compile GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Hyperparameters
batch_size = 64
epochs = 1000
half_batch = batch_size // 2

# 6. KFold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fungsi untuk training AnoGAN dengan KFold
for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
    print(f"Training fold {fold + 1}")

    # Split data berdasarkan KFold
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Training loop untuk satu fold
    for epoch in range(epochs):
        # Latih discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_data = X_train[idx]

        # Sampel noise untuk data palsu dari generator
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_data = generator.predict(noise)

        # Label untuk data asli dan palsu
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Latih diskriminator dengan data asli dan palsu
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)

        # Latih generator (membekukan diskriminator untuk sementara)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))  # Generator mencoba menghasilkan data yang 'valid'
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Tampilkan kemajuan pelatihan setiap 100 epoch
        if epoch % 100 == 0:
            print(f"Fold {fold + 1} - Epoch {epoch}/{epochs} [D loss: {d_loss_real[0]}] [G loss: {g_loss}]")

    # Setelah training, lakukan prediksi dan deteksi anomali
    # Menghitung error rekonstruksi untuk mendeteksi anomali
    reconstructed_data = generator.predict(np.random.normal(0, 1, (X_val.shape[0], latent_dim)))
    reconstruction_error = np.mean(np.abs(reconstructed_data - X_val), axis=1)

    # Menentukan threshold untuk mendeteksi anomali (misalnya menggunakan quantile atau manual)
    threshold = np.percentile(reconstruction_error, 95)  # ambil threshold 95% dari error rekonstruksi

    # Deteksi anomali
    y_pred = (reconstruction_error > threshold).astype(int)

    # Evaluasi performa model pada fold ini
    print(f"Evaluasi Fold {fold + 1}:")
    print(classification_report(y_val, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Training fold 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Fold 1 - Epoch 0/1000 [D loss: 0.7537317276000977] [G loss: [array(0.7162057, dtype=float32), array(0.7162057, dtype=float32), array(0.578125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 